In [3]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
imageLocation = bs.find('a', {'id': 'logo'}).find('img')['src']
urlretrieve (imageLocation, 'logo.jpg')

('logo.jpg', <http.client.HTTPMessage at 0x1119cacf8>)

In [2]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

downloadDirectory = 'downloaded'
baseUrl = 'http://pythonscraping.com'

def getAbsoluteURL(baseUrl, source):
    if source.startswith('http://www.'):
        url = 'http://{}'.format(source[11:])
    elif source.startswith('http://'):
        url = source
    elif source.startswith('www.'):
        url = source[4:]
        url = 'http://{}'.format(source)
    else:
        url = '{}/{}'.format(baseUrl, source)
    if baseUrl not in url:
        return None
    return url

def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory):
    path = absoluteUrl.replace('www.', '')
    path = path.replace(baseUrl, '')
    path = downloadDirectory+path
    directory = os.path.dirname(path)

    if not os.path.exists(directory):
        os.makedirs(directory)

    return path

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src=True)

for download in downloadList:
    fileUrl = getAbsoluteURL(baseUrl, download['src'])
    if fileUrl is not None:
        print(fileUrl)

urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, downloadDirectory))

http://pythonscraping.com/misc/jquery.js?v=1.4.4
http://pythonscraping.com/misc/jquery.once.js?v=1.2
http://pythonscraping.com/misc/drupal.js?os2esm
http://pythonscraping.com/sites/all/themes/skeletontheme/js/jquery.mobilemenu.js?os2esm
http://pythonscraping.com/sites/all/modules/google_analytics/googleanalytics.js?os2esm
http://pythonscraping.com/sites/default/files/lrg_0.jpg
http://pythonscraping.com/img/lrg%20(1).jpg


('downloaded/img/lrg%20(1).jpg', <http.client.HTTPMessage at 0x107b827f0>)

In [4]:
import csv # 파이썬 데이터를 csv에 저장하기 위함 

csvFile = open('test.csv', 'w+') # 경로가 따로 없으므로, 내가 작업하고 있는 폴더 상에 생성됨. open() : 더블클릭
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number plus 2', 'number times 2')) # writerow() : row 한 줄에 이것(담는 내용은 하나로 묶어줌)을 쓰고 개행
    for i in range(10):
        writer.writerow( (i, i+2, i*2))
finally:
    csvFile.close()

In [4]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 크롤링한 데이터를 csv 파일에 저장하는 코드

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
# The main comparison table is currently the first table on the page
table = bs.findAll('table',{'class':'wikitable'})[0]
rows = table.findAll('tr')
'''
table은 사실상 csv로 저장하기 가장 좋은 형태. 
애초에 형태 자체가 똑같으므로 어떤 방식으로 저장할지에 대한 고민 없이 그대로 긁어서 저장하면 됨.
'''

csvFile = open('editors.csv', 'wt+', encoding='UTF-8') # 인코딩 방식 때문에 에러나는 것 해결 위해 encoding='UTF-8' 추가
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']): # th : tr의 인덱스같은 존재, td : tr 안의 데이터
            csvRow.append(cell.get_text())
        writer.writerow(csvRow) # editors.csv에서 하나의 row에 csvRow 써줌
finally: # 예외처리 try-except-finally : try문 수행 도중 예외가 발생했든 아니든(try문 수행하든 except문 수행하든) finally문 항상 수행
    csvFile.close()

In [5]:
## MySQL PYTHON 연동

from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql # 파이썬과 MySQL 연동 => 서로 데이터 주고받을 수 있음. 
import re

conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock', # unix_socket='/tmp/mysql.sock' 삭제하고 passwd 내 mysql 비밀번호로 바꿔줘야 함
                       user='root', passwd='root', db='mysql', charset='utf8')
cur = conn.cursor() # 파이썬에서 긁어온 데이터를 MySQL 데이터베이스로 넘겨주기 위한 객체
cur.execute('USE scraping') # USE scraping : mysql에서 쓰는 쿼리문. scraping 사용하기 위해 mysql로 들어가는 쿼리문. 

random.seed(datetime.datetime.now())

def store(title, content):
    cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content)) # 이 부분은 cowork에서 다른 팀원들이 코드 어떻게 짜든 상관 없는 부분 (github의 add같은 느낌..?)
        # INSERT INTO pages (title, content) VALUES ("%s", "%s") : mysql 쿼리문. => pages 테이블의 title, content 컬럼에다가 각각 해당 %s값을 넣어주겠다. 
    cur.connection.commit() # 실제로 데이터베이스에 써넣는 행위 (github commit과 같은 개념으로 log가 계속 누적됨.)

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org'+articleUrl)
    bs = BeautifulSoup(html, 'html.parser') # urlopen으로 불러오면 html만 넣어줘도 됨
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$')) # html 안의 모든 내부링크 찾아서 리턴

links = getLinks('/wiki/Kevin_Bacon')
try:
    while len(links) > 0: # 랜덤으로 들어간 어떤 웹페이지가 내부 링크를 갖지 않을 때까지
         newArticle = links[random.randint(0, len(links)-1)].attrs['href'] # 전체 a태그 중 균일분포 랜덤으로 하나 뽑고 href 속성값(링크) 담음
         print(newArticle) # 뽑은 링크 출력
         links = getLinks(newArticle) # 이 코드를 통해 계속 내부링크로 타고 들어감
finally:
    cur.close()
    conn.close()


InternalError: (1049, "Unknown database 'scraping'")

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pymysql # 파이썬과 DB를 연결시켜주는 패키지
from random import shuffle

conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
                       user='root', passwd='root', db='mysql', charset='utf8')
cur = conn.cursor() # sql문을 실행시키고 결과를 얻어올 때 사용할 커서(cursor) 만듦
cur.execute('USE wikipedia') # sql문을 실행(execute), execute : 파이썬에서 mysql로 쿼리문을 던져주는 함수

def insertPageIfNotExists(url):
    cur.execute('SELECT * FROM pages WHERE url = %s', (url)) # pages 테이블에서 해당 url을 가진 모든 row 가져옴
    if cur.rowcount == 0: # 하나도 없으면 (DB가 파이썬 변수 rowcount에 0을 담아줌)
        cur.execute('INSERT INTO pages (url) VALUES (%s)', (url)) # pages 테이블에 해당 url을 가진 새로운 추가
        conn.commit() # 실제로 데이터베이스에 써줌
        
        test = cur.lastrowid
        print('lastrowid:', test) # 새로운 페이지에 들어올 때마다 실행되므로 1씩 증가되며 출력
        return test # db_cursor를 이용해 excute한 테이블의 마지막 row id 값을 가져옴 => 지금 if문 안에서는 무조건 0이 리턴됨
    else: # 있으면
        test1 = cur.fetchone()[0]
        print('fetchone()[0]:', test1) # 지금 excute한 url이 pages 테이블 상의 어떤 row에 존재하는지 출력 => 당연히 중복된 값 출력 가능
        return test1 # db_cursor를 이용해 excute한 쿼리문에 해당하는 한 줄의 row을 읽어서 0번째 column값을 리턴 -> id 리턴
                                 # 즉, MySQL 에 직접 SELECT * FROM pages WHERE url = .. 쿼리문을 입력하면 출력되는 row를 파이썬으로 가져옴
        '''
        cur.fetchall() : 모든 데이터를 한꺼번에 가져올 때 사용
        cur.fetchone() : 1개의 데이터를 가져올 때 사용. 
        cur.fetchmany(n) : n개 만큼의 데이터를 한꺼번에 가져올 때 사용
        
        연속으로 실행 시, 마지막으로 읽어들인 데이터의 다음 데이터를 자동으로 가져온다.
            ex_  5개 row가 있다면
            print(cur.fetchmany(2))
            print(cur.fetchone())
            print(cur.fetchall())
            --------------------------------------
            [(1, '/wiki/Kevin_Bacon', '2021-07-05 10:26:43'), (2, '/wiki/Kevin_Bacon_(disambiguation)', '2021-07-05 10:26:44')]
            (3, '/wiki/Kevin_Bacon_(producer)', '2021-07-05 10:26:45')
            [(4, '/wiki/Rotherham', '2021-07-05 10:26:46'), (5, '/wiki/Rotherham_(disambiguation)', '2021-07-05 10:26:47')]
        '''
def loadPages():
    cur.execute('SELECT * FROM pages')
    pages = [row[1] for row in cur.fetchall()] # pages에 담긴 모든 row에 대해 url 가져와서 리스트에 담음
    return pages

def insertLink(fromPageId, toPageId):
    cur.execute('SELECT * FROM links WHERE fromPageId = %s AND toPageId = %s', 
                  (int(fromPageId), int(toPageId))) # links 테이블에서 해당 fromPageId, toPageId를 가진 모든 row 가져옴
    if cur.rowcount == 0: # 하나도 없으면
        cur.execute('INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)', 
                    (int(fromPageId), int(toPageId))) # links 테이블에 해당 fromPageId, toPageId를 갖는 새로운 row 추가
        conn.commit()
def pageHasLinks(pageId):
    cur.execute('SELECT * FROM links WHERE fromPageId = %s', (int(pageId))) # links 테이블에서 해당 pageId를 fromPageId로 가진 모든 row 가져옴
    rowcount = cur.rowcount
    if rowcount == 0: # 하나도 없으면
        return False
    return True # 하나라도 있으면

def getLinks(pageUrl, recursionLevel, pages): # pageUrl : 현재 웹페이지 주소, pages : pages 테이블에 담긴 모든 url 담은 리스트
    if recursionLevel > 4: # 재귀 5번 돌고 6번째 끝남. 
        return

    pageId = insertPageIfNotExists(pageUrl) # pages에서 pageUrl을 url로 가진 첫 번째 row에 담긴 id 가져옴
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser') # urlopen으로 열면 html만 넣어줘도 됨
    links = bs.findAll('a', href=re.compile('^(/wiki/)((?!:).)*$')) # html 안의 모든 내부링크 가진 a태그 찾아서 리턴
    links = [link.attrs['href'] for link in links] # href 속성값만 빼서 담음

    for link in links: # link(href 속성값) 하나씩
        linkId = insertPageIfNotExists(link) # pages 테이블에서 link를 url로 가진 첫 번째 row에 담긴 id 가져옴
        insertLink(pageId, linkId) # links 테이블에 pageId, linkId를 각각 fromPageId, toPageId로 갖는 row가 없다면 추가. 
                                   # pages 테이블에 담긴 페이지들 간의 연결관계를 links 테이블에 추가. 즉 어떤 페이지에서 어떤 페이지로 연결되고 있는지 페이지 id로 각각 나타냄. 
        if not pageHasLinks(linkId): # links 테이블에 linkId(toPageId)를 fromPageId로 갖는 row가 있을 때, 즉 이미 한 번 들려서 pageUrl이었던 적 있는 url을 다른 페이지의 내부 링크로 또 만날 때
                                     # 재귀로 인해 그 페이지로 다시 들어가서 무한 루프로 빠지면 안 되기 때문에 if문 못 들어오도록 걸러줌.
            print("PAGE HAS NO LINKS: {}".format(link))
            pages.append(link) # pages 테이블에 담긴 모든 url 담은 리스트에 link도 추가
            getLinks(link, recursionLevel+1, pages) # 재귀→계속 첫 번째 내부링크로만 들어감
        
        
getLinks('/wiki/Kevin_Bacon', 0, loadPages()) # 처음엔 loadPages() : []
'''
PAGE HAS NO LINKS: /wiki/Philadelphia,_Pennsylvania
PAGE HAS NO LINKS: /wiki/Philly_(disambiguation)
PAGE HAS NO LINKS: /wiki/Philadelphia
PAGE HAS NO LINKS: /wiki/Philadelphia_(disambiguation)
PAGE HAS NO LINKS: /wiki/Philadelphia_County,_Pennsylvania
'''
cur.close()
conn.close()

AttributeError: module 'socket' has no attribute 'AF_UNIX'

In [ ]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('The body of the email is here')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'ryan@pythonscraping.com'
msg['To'] = 'webmaster@pythonscraping.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

In [1]:
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

def sendMail(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] ='christmas_alerts@pythonscraping.com'
    msg['To'] = 'ryan@pythonscraping.com'

    s = smtplib.SMTP('localhost')
    s.send_message(msg)
    s.quit()

bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
while(bs.find('a', {'id':'answer'}).attrs['title'] == 'NO'):
    print('It is not Christmas yet.')
    time.sleep(3600)
    bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
sendMail('It\'s Christmas!', 
         'According to http://itischristmas.com, it is Christmas!')

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
'